In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, RandomSampler
import numpy
import math
import numbers

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [58]:
class GaussianSmoothing(nn.Module):
    """
    Apply gaussian smoothing on a
    1d, 2d or 3d tensor. Filtering is performed seperately for each channel
    in the input using a depthwise convolution.
    Arguments:
        channels (int, sequence): Number of channels of the input tensors. Output will
            have this number of channels as well.
        kernel_size (int, sequence): Size of the gaussian kernel.
        sigma (float, sequence): Standard deviation of the gaussian kernel.
        dim (int, optional): The number of dimensions of the data.
            Default value is 2 (spatial).
    """
    def __init__(self, channels, kernel_size, sigma, dim=2):
        super(GaussianSmoothing, self).__init__()
        if isinstance(kernel_size, numbers.Number):
            kernel_size = [kernel_size] * dim
        if isinstance(sigma, numbers.Number):
            sigma = [sigma] * dim

        # The gaussian kernel is the product of the
        # gaussian function of each dimension.
        kernel = 1
        meshgrids = torch.meshgrid(
            [
                torch.arange(size, dtype=torch.float32)
                for size in kernel_size
            ]
        )
        for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
            mean = (size - 1) / 2
            kernel *= 1 / (std * math.sqrt(2 * math.pi)) * \
                      torch.exp(-((mgrid - mean) / std) ** 2 / 2)

        # Make sure sum of values in gaussian kernel equals 1.
        kernel = kernel / torch.sum(kernel)

        # Reshape to depthwise convolutional weight
        kernel = kernel.view(1, 1, *kernel.size())
        kernel = kernel.repeat(channels, *[1] * (kernel.dim() - 1))

        self.register_buffer('weight', kernel)
        self.groups = channels

        if dim == 1:
            self.conv = F.conv1d
        elif dim == 2:
            self.conv = F.conv2d
        elif dim == 3:
            self.conv = F.conv3d
        else:
            raise RuntimeError(
                'Only 1, 2 and 3 dimensions are supported. Received {}.'.format(dim)
            )

    def forward(self, input):
        """
        Apply gaussian filter to input.
        Arguments:
            input (torch.Tensor): Input to apply gaussian filter on.
        Returns:
            filtered (torch.Tensor): Filtered output.
        """
        return self.conv(input, weight=self.weight, groups=self.groups)


In [63]:
class akCNN(nn.Module):
    def __init__(self):
        super(akCNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 128, 3, padding=1),nn.ELU())
        # modify kernel size here
        self.smoothing = GaussianSmoothing(128, kernel_size=3, sigma=1)

        self.stack1 = nn.Sequential(
        nn.Conv2d(128, 128, 3),
        nn.ELU(),
        nn.MaxPool2d(2),
        nn.Dropout2d(p=0.1)
        )
    
        self.stack2 = nn.Sequential(
        nn.Conv2d(128, 256, 3, padding=1),
        nn.ELU(),
        nn.Conv2d(256, 256, 3),
        nn.ELU(),
        nn.MaxPool2d(2),
        nn.Dropout2d(p=0.25)
        )
        
        self.stack3 = nn.Sequential(
        nn.Conv2d(256, 512, 3, padding=1),
        nn.ELU(),
        nn.Conv2d(512, 512, 3),
        nn.ELU(),
        nn.MaxPool2d(2),
        nn.Dropout2d(p=0.5)
        )
    
        self.last = nn.Sequential(
        nn.Linear(2048,1024),
        nn.ELU(),
        nn.Dropout2d(p=0.5),
        nn.Linear(1024,100),
        nn.Softmax(dim=1)
        )
    
        
    def forward(self, x, verbose=False):
        x = self.conv1(x)
#         if self.training and self.sigma != 0:
#             scale = self.sigma * x
#             sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
#         x = x + sampled_noise
        x = self.smoothing(x)
        x = self.stack1(x)
        x = self.stack2(x)
        x = self.stack3(x)
        x = torch.flatten(x,1)
        x = self.last(x)
        return x

In [39]:
accuracy_list = []

def train(model, device, train_loader, optimizer, epoch, log_interval = 100):
    # Set model to training mode
    model.train()
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):    
        # Send data and target to device
        data, target = data.to(device), target.to(device)
        
        # Zero out the optimizer
        optimizer.zero_grad()
        # Pass data through model
        output = model(data)
        # Compute the negative log likelihood loss
        loss = F.nll_loss(output, target)
        # Backpropagate loss
        loss.backward()
        
        # Make a step with the optimizer
        optimizer.step()
        
        # Print loss (uncomment lines below once implemented)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
# Define test method
def test(model, device, test_loader):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    # don't need autograd for eval
    with torch.no_grad():
        # Loop through data points
        for data, target in test_loader:
            # Send data to device
            data, target = data.to(device), target.to(device)
            # Pass data through model
            output = model(data)
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            # sum losses over minibatch
            test_loss += F.nll_loss(output, target, reduction = 'sum').item()
            # Get predictions from the model for each data point
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
                    
            # Add number of correct predictions to total num_correct 
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    
    # Compute the average test_loss
    avg_test_loss = test_loss / len(test_loader.dataset)
    
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

In [36]:
# Download data
trans = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                            ])
cifar100Train = datasets.CIFAR100('data', train=True, transform=trans, download=True)
trainSampler = RandomSampler(cifar100Train)
train_loader = DataLoader(cifar100Train, batch_size=32, sampler=trainSampler)
cifar100test = datasets.CIFAR100('data', train=False, transform=trans, download=True)
testSampler = RandomSampler(cifar100test)
test_loader = DataLoader(cifar100test, batch_size=32, sampler=testSampler)

In [ ]:
# Define test method
def test(model, device, test_loader):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    # don't need autograd for eval
    with torch.no_grad():
        # Loop through data points
        for data, target in test_loader:        
            # Send data to device
            data, target = data.to(device), target.to(device)
            # Pass data through model
            output = model(data)
            
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            # sum losses over minibatch
            test_loss += F.nll_loss(output, target, reduction = 'sum').item()
            
            # Get predictions from the model for each data point
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
                    
            # Add number of correct predictions to total num_correct 
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    
    # Compute the average test_loss
    avg_test_loss = test_loss / len(test_loader.dataset)
    
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

In [ ]:
# Define model and sent to device
'''
    sigma (float, optional): relative standard deviation used to generate the
    noise. Relative means that it will be multiplied by the magnitude of
    the value your are adding the noise to. This means that sigma can be
    the same regardless of the scale of the vector.
'''
            
model = akCNN()

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    # Train model
    train(model, device, train_loader, optimizer, epoch)
    break
    # Test model
    test(model, device, test_loader)

In [64]:
fakeInputs = torch.rand(16, 3, 32, 32).to(device)
net = akCNN()
net.to(device)
net(fakeInputs)

tensor([[0.0100, 0.0102, 0.0099,  ..., 0.0098, 0.0097, 0.0101],
        [0.0097, 0.0099, 0.0099,  ..., 0.0101, 0.0102, 0.0099],
        [0.0099, 0.0102, 0.0100,  ..., 0.0098, 0.0098, 0.0099],
        ...,
        [0.0100, 0.0102, 0.0099,  ..., 0.0097, 0.0102, 0.0102],
        [0.0100, 0.0102, 0.0101,  ..., 0.0099, 0.0100, 0.0100],
        [0.0101, 0.0101, 0.0100,  ..., 0.0098, 0.0102, 0.0101]],
       grad_fn=<SoftmaxBackward>)